In [2]:
import scipy.sparse as sps
from Recommenders.Hybrid.ItemUserHybridKNNRecommender import ItemUserHybridKNNRecommender
from Utils.Evaluator import EvaluatorHoldout
import Utils.Reader as Reader
from Data_manager.split_functions.split_train_validation_random_holdout import\
    split_train_in_two_percentage_global_sample

In [4]:
#FACTORIZED SPLIT
URM_all,ICM_all=Reader.load_URM_and_ICM_users("../data/interactions_and_impressions.csv","../data/rewatches.csv",ICM_vals_to_not_keep=[0.0],URM_values_to_replace={0:0.2})
cutoff_list=[ 10]

URM_train, URM_test = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.8)
URM_train, URM_validation = split_train_in_two_percentage_global_sample(URM_train, train_percentage = 0.8)

evaluator_validation = EvaluatorHoldout(URM_validation, cutoff_list=cutoff_list)
evaluator_test = EvaluatorHoldout(URM_test, cutoff_list=cutoff_list)
'''
profile_length = np.ediff1d(URM_all.indptr)
block_size = int(len(profile_length)*0.25)

start_lower = 0
end_lower = 3*block_size
end_higher = len(profile_length)
sorted_users = np.argsort(profile_length)

users_in_lower = set(sorted_users[0:end_lower])
'''
#ICM_train, ICM_test = split_train_in_two_percentage_global_sample(ICM_all ,train_percentage=0.80)

stacked_ICM = Reader.combine(ICM=ICM_all, URM = URM_train)
stacked_URM=stacked_ICM.T
stacked_URM

Number of tags	 631, Number of item-tag tuples 1051828
Unique UserID in the URM are 41629
Unique UserID in the URM and ICM are 41629
Unique ItemID in the URM are 24507
Unique ItemID in the URM and ICM are 24507
Unique FeatureID in the URM are 631
EvaluatorHoldout: Ignoring 766 ( 1.8%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 289 ( 0.7%) Users that have less than 1 test interactions


<42260x24507 sparse matrix of type '<class 'numpy.float64'>'
	with 1075984 stored elements in Compressed Sparse Column format>

In [6]:

from Recommenders.RP3betaCBFRecommender import RP3betaCBFRecommender

rp3betaCBF_recommender = RP3betaCBFRecommender(
    URM_train=URM_train,
    ICM_train=stacked_ICM,
    verbose=False
)

rp3betaCBF_recommender.fit(
    topK=int(529.1628484087545),
    alpha=0.45304737831676245,
    beta=0.226647894170121,
    implicit=False
)

result_df, _ = evaluator_validation.evaluateRecommender(rp3betaCBF_recommender)
result_df["MAP"]


/Users/fasa/PycharmProjects/RecSys-Challenge-2022-Polimi/Utils/metrics.py:276: RuntimeWarning: overflow encountered in power
  return np.sum(np.divide(np.power(2, scores) - 1, np.log2(np.arange(scores.shape[0], dtype=np.float64) + 2)),
/Users/fasa/PycharmProjects/RecSys-Challenge-2022-Polimi/Utils/metrics.py:270: RuntimeWarning: invalid value encountered in double_scalars
  ndcg_ = rank_dcg / ideal_dcg


EvaluatorHoldout: Processed 40863 (100.0%) in 52.14 sec. Users per second: 784


cutoff
10    0.015569
Name: MAP, dtype: object

In [ ]:
import itertools
from tqdm.notebook import tqdm
# GRID SEARCH RP3BETA
best=0
config={
    "k":0,
    "a":0,
    "b":0

}
from Recommenders.RP3betaCBFRecommender import RP3betaCBFRecommender

rp3betaCBF_recommender = RP3betaCBFRecommender(
    URM_train=URM_train,
    ICM_train=stacked_ICM,
    verbose=False
)

TopkList=[100,125,150,175,200,225,250,275]
aList=[0.1,0.125,0.150,0.175,0.2,0.225,0.250,0.275]
bList=[0.3,0.325,0.350,0.375,0.4,0.425,0.450,0.475]
for x in tqdm(itertools.product(TopkList,aList,bList),total=int(len(TopkList)*len(aList)*len(bList)), position=0, leave=True):
    k=x[0]
    a=x[1]
    b=x[2]
    rp3betaCBF_recommender.fit(
        topK=k,
        alpha=a,
        beta=b,
        implicit=False
    )

    result_df, _ = evaluator_validation.evaluateRecommender(rp3betaCBF_recommender)
    if result_df["MAP"][10] > best:
        best=result_df["MAP"][10]
        config={
            "k":k,
            "a":a,
            "b":b
        }
        print("NEW BEST: "+str(best))
        print("Config:"+str(config))

best,config